In [1]:
!git clone https://github.com/JJGO/UniverSeg
!python -m pip install -r ./UniverSeg/requirements.txt

fatal: destination path 'UniverSeg' already exists and is not an empty directory.


In [1]:
import cactas as C

In [2]:
import math
import itertools

from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

import einops as E

In [3]:
import sys
sys.path.append('UniverSeg')

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from universeg import universeg
model = universeg(pretrained=True)
_ = model.to(device)

In [4]:
DATAPATH='/raid/mpsych/CACTAS/DATA/ESUS'

In [5]:
images, labels = C.Helper.load_data(DATAPATH)

In [6]:
X_train, y_train, X_test, y_test = C.Helper.split_patients(images, labels)

In [7]:
X_train, y_train, X_test, y_test = C.Helper.normalization(DATAPATH, X_train, y_train, X_test, y_test)

In [8]:
slice_to_patient_mapping, last_key = C.Helper.map_and_key(y_train)

In [9]:
X_train, y_train, X_test, y_test = C.Helper.extract_slices(X_train, y_train, X_test, y_test)

(13273, 512, 512, 1) (13273, 512, 512, 1) (3493, 512, 512, 1) (3493, 512, 512, 1)


In [10]:
X_train, y_train = C.Helper.filter_slices(X_train, y_train)

(575, 512, 512, 1) (575, 512, 512, 1)


In [11]:
X_train_torch = [torch.from_numpy(arr) for arr in X_train]
stacked_tensor = torch.stack(X_train_torch, dim=0)

In [12]:
y_train_torch = [torch.from_numpy(arr) for arr in y_train]
stacked_tensor2 = torch.stack(y_train_torch, dim=0)

In [13]:
X_test_torch = [torch.from_numpy(arr) for arr in X_test]
stacked_tensor3 = torch.stack(X_test_torch, dim=0)

In [14]:
y_test_torch = [torch.from_numpy(arr) for arr in y_test]
stacked_tensor4 = torch.stack(y_test_torch, dim=0)

In [16]:
stacked_tensor.shape

torch.Size([575, 512, 512, 1])

In [18]:
X_train_t = stacked_tensor.permute(0, 3, 1, 2)
y_train_t = stacked_tensor2.permute(0, 3, 1, 2)

In [20]:
X_test_t = stacked_tensor3.permute(0, 3, 1, 2)
y_test_t = stacked_tensor4.permute(0, 3, 1, 2)

In [32]:
# run inference
logits = model(X_test_t[None], X_train_t[None], y_train_t[None])[0].to('cpu')
pred = torch.sigmoid(logits)

EinopsError:  Error while processing repeat-reduction pattern "B Sy Cy H W -> B Sx Sy Cy H W".
 Input tensor shape: torch.Size([575, 1, 1024, 512]). Additional info: {'Sx': 1}.
 Wrong shape: expected 5 dims. Received 4-dim tensor.